<a href="https://colab.research.google.com/github/nithindsouza/Deeper-Attention-Based-Network-for-Structured-Data-DAN/blob/main/DAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Deeper Attention-Based Network for Structured Data (DAN)**
This Notebook is about DAN for Cretio dataset (kaggle)

[View Paper](https://link.springer.com/chapter/10.1007%2F978-981-15-7981-3_17) 

[Dataset Link](https://www.kaggle.com/c/criteo-display-ad-challenge/data)

In [ ]:
!wget -c --no-check-certificate https://labs.criteo.com/wp-content/uploads/2015/04/dac_sample.tar.gz -O - | tar -xz

--2021-09-30 06:08:32--  https://labs.criteo.com/wp-content/uploads/2015/04/dac_sample.tar.gz
Resolving labs.criteo.com (labs.criteo.com)... 87.98.154.146
Connecting to labs.criteo.com (labs.criteo.com)|87.98.154.146|:443... connected.
  Issued certificate has expired.
HTTP request sent, awaiting response... 200 OK
Length: 8787154 (8.4M) [application/x-gzip]
Saving to: ‘STDOUT’

-                     0%[                    ]       0  --.-KB/s               tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
tar: Ignoring unknown extended header keyword 'SCHILY.dev'
tar: Ignoring unknown extended header keyword 'SCHILY.ino'
tar: Ignoring unknown extended header keyword 'SCHILY.nlink'
-                   100%[===================>]   8.38M  7.40MB/s    in 1.1s    

2021-09-30 06:08:34 (7.40 MB/s) - written to stdout [8787154/8787154]

tar: Ignoring unknown extended header keyword 'SCHILY.dev'
tar: Ignoring unknown extended header keyword 'SCHILY.ino'
tar: Ignoring unkn

### Required Packages

In [ ]:
from keras.layers import Input,Embedding,Concatenate,Dense,Dropout,Reshape
from tensorflow.keras import layers
from sklearn import preprocessing
from tensorflow import keras
import tensorflow as tf
import pandas as pd
import numpy as np

### Load and preprocess the data

In [ ]:
# Load the dataset and fill the nan with 0
columns = ['label', *(f'I{i}' for i in range(1, 14)), *(f'C{i}' for i in range(1, 27))]
df = pd.read_csv('dac_sample.txt', sep='\t', names=columns).fillna(0)
df.tail(5)

,label,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,I11,I12,I13,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
99995,0,1.0,60,37.0,0.0,1.0,0.0,4.0,0.0,23.0,1.0,3.0,0.0,0.0,68fd1e64,91381efc,eacb18f6,f00be896,25c83c98,fe6b92e5,2489e185,985e3fcb,a73ee510,3b08e48b,4d1f7d97,d6b4fe71,954029f8,1adce6ef,a0602981,2a08cb76,e5ba7672,38b82d9f,21ddcdc9,b1252a9d,24e5131b,0,32c7478e,4fa16304,47907db5,72966777
99996,1,0.0,0,12.0,0.0,173121.0,0.0,0.0,3.0,10.0,0.0,0.0,0.0,0.0,68fd1e64,f0cf0024,6f67f7e5,41274cd7,4cf72387,fbad5c96,9e5d3694,0b153874,7cc72ec2,be289b53,699034a0,623049e6,a0edec24,1adce6ef,55dc357b,c92f3b61,e5ba7672,b04e4670,21ddcdc9,b1252a9d,60f6221e,0,32c7478e,43f13e8b,ea9a246c,731c3655
99997,0,10.0,2,1.0,26.0,482.0,60.0,10.0,11.0,60.0,1.0,1.0,0.0,59.0,8cf07265,c8687797,b063fe4e,4b972461,25c83c98,7e0ccccf,176dc88e,d7c4a8f5,a73ee510,3b08e48b,d2b7c44b,8cdc4941,68637c0d,b28479f6,dc96c4b0,3084c78b,07c540c4,a7e06874,21ddcdc9,a458ea53,514b7308,0,32c7478e,2fd70e1c,010f6491,ec26ad35
99998,0,0.0,390,43.0,4.0,345365.0,0.0,0.0,4.0,4.0,0.0,0.0,0.0,4.0,0e78bd46,a796837e,dffca8ba,0fa0d423,25c83c98,fe6b92e5,bc324536,51d76abe,7cc72ec2,5cad6330,2bcfb78f,93bab460,e6fc496d,cfef1c29,f0bf9094,6bb29970,e5ba7672,1cdbd1c5,0,0,d9d9202f,0,32c7478e,8fc66e78,0,0
99999,0,0.0,-1,137.0,19.0,9504.0,0.0,0.0,22.0,22.0,0.0,0.0,0.0,22.0,05db9164,38d50e09,dd17c91c,82a61820,25c83c98,7e0ccccf,a088b320,5b392875,a73ee510,323dec10,6be5122d,75529ad8,c8c105ca,07d13a8f,ee569ce2,5eea53aa,e5ba7672,582152eb,21ddcdc9,5840adea,0f78ab39,c9d4222a,32c7478e,cafb4e4d,001f3601,99f4f64c


In [ ]:
# Standardize Numerical Features
num_cols = [c for c in columns if 'I' in c]
df[num_cols] = preprocessing.StandardScaler().fit_transform(df[num_cols])
df.tail(5)

,label,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,I11,I12,I13,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
99995,0,-0.136628,-0.131659,0.008328,-0.649411,-0.260475,-0.319543,-0.164177,-0.291503,-0.343200,1.108068,0.156775,-0.083099,-0.199103,68fd1e64,91381efc,eacb18f6,f00be896,25c83c98,fe6b92e5,2489e185,985e3fcb,a73ee510,3b08e48b,4d1f7d97,d6b4fe71,954029f8,1adce6ef,a0602981,2a08cb76,e5ba7672,38b82d9f,21ddcdc9,b1252a9d,24e5131b,0,32c7478e,4fa16304,47907db5,72966777
99996,1,-0.261412,-0.281091,-0.043230,-0.649411,2.430998,-0.319543,-0.226698,-0.227013,-0.389491,-0.582864,-0.502869,-0.083099,-0.199103,68fd1e64,f0cf0024,6f67f7e5,41274cd7,4cf72387,fbad5c96,9e5d3694,0b153874,7cc72ec2,be289b53,699034a0,623049e6,a0edec24,1adce6ef,55dc357b,c92f3b61,e5ba7672,b04e4670,21ddcdc9,b1252a9d,60f6221e,0,32c7478e,43f13e8b,ea9a246c,731c3655
99997,0,0.986427,-0.276110,-0.065915,1.884746,-0.252997,-0.136274,-0.070395,-0.055038,-0.211450,1.108068,-0.282987,-0.083099,1.058586,8cf07265,c8687797,b063fe4e,4b972461,25c83c98,7e0ccccf,176dc88e,d7c4a8f5,a73ee510,3b08e48b,d2b7c44b,8cdc4941,68637c0d,b28479f6,dc96c4b0,3084c78b,07c540c4,a7e06874,21ddcdc9,a458ea53,514b7308,0,32c7478e,2fd70e1c,010f6491,ec26ad35
99998,0,-0.261412,0.690217,0.020702,-0.259541,5.108853,-0.319543,-0.226698,-0.205516,-0.410855,-0.582864,-0.502869,-0.083099,-0.113836,0e78bd46,a796837e,dffca8ba,0fa0d423,25c83c98,fe6b92e5,bc324536,51d76abe,7cc72ec2,5cad6330,2bcfb78f,93bab460,e6fc496d,cfef1c29,f0bf9094,6bb29970,e5ba7672,1cdbd1c5,0,0,d9d9202f,0,32c7478e,8fc66e78,0,0
99999,0,-0.261412,-0.283581,0.214559,1.202473,-0.112733,-0.319543,-0.226698,0.181426,-0.346761,-0.582864,-0.502869,-0.083099,0.269866,05db9164,38d50e09,dd17c91c,82a61820,25c83c98,7e0ccccf,a088b320,5b392875,a73ee510,323dec10,6be5122d,75529ad8,c8c105ca,07d13a8f,ee569ce2,5eea53aa,e5ba7672,582152eb,21ddcdc9,5840adea,0f78ab39,c9d4222a,32c7478e,cafb4e4d,001f3601,99f4f64c


In [ ]:
# Label encoding Categorical Features
cat_cols = [c for c in columns if 'C' in c]
mappings = {
    col: dict(zip(values, range(len(values))))
    for col, values in map(lambda col: (col, df[col].unique()), cat_cols)
}
for col, mapping in mappings.items():
    df[col] = df[col].map(mapping.get)
df.tail(5)

,label,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,I11,I12,I13,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
99995,0,-0.136628,-0.131659,0.008328,-0.649411,-0.260475,-0.319543,-0.164177,-0.291503,-0.343200,1.108068,0.156775,-0.083099,-0.199103,0,303,13120,8673,0,1,4113,7,0,2,2477,12477,2018,0,1143,10794,0,717,0,0,11793,0,1,4973,16,3841
99996,1,-0.261412,-0.281091,-0.043230,-0.649411,2.430998,-0.319543,-0.226698,-0.227013,-0.389491,-0.582864,-0.502869,-0.083099,-0.199103,0,1,1,1,3,5,2579,1,1,1252,1177,1,1027,0,17,1,0,1,0,0,1,0,1,1,2,1
99997,0,0.986427,-0.276110,-0.065915,1.884746,-0.252997,-0.136274,-0.070395,-0.055038,-0.211450,1.108068,-0.282987,-0.083099,1.058586,2,113,3287,2521,0,0,2231,47,0,2,554,3160,508,1,657,2874,1,176,0,3,3024,0,1,1649,6,3212
99998,0,-0.261412,0.690217,0.020702,-0.259541,5.108853,-0.319543,-0.226698,-0.205516,-0.410855,-0.582864,-0.502869,-0.083099,-0.113836,66,93,318,274,0,1,764,14,1,3450,157,309,149,7,284,295,0,210,1,2,301,0,1,120,1,2
99999,0,-0.261412,-0.283581,0.214559,1.202473,-0.112733,-0.319543,-0.226698,0.181426,-0.346761,-0.582864,-0.502869,-0.083099,0.269866,3,14,108,72,0,0,121,5,0,8645,115,108,112,2,15,105,0,15,0,1,108,3,1,59,4,76


In [ ]:
def auc(y_true, y_pred):
    def fallback_auc(y_true, y_pred):
        try:
            return metrics.roc_auc_score(y_true, y_pred)
        except:
            return 0.5
    return tf.py_function(fallback_auc, (y_true, y_pred), tf.double)

In [ ]:
#splitting data
import math
size = df.shape[0]
train = df[: math.floor(0.8 * size)]
test = df[math.floor(0.8 * size) :]
print('Train Shape:',train.shape[0],'\nTest Shape:',test.shape[0])
train.to_csv('train.csv', index=False)
test.to_csv('test.csv', index=False)

Train Shape: 80000 
Test Shape: 20000


In [ ]:
#preparing categorical features
input_train_list = []
input_test_list = []
for c in cat_cols:
  input_train_list.append(train[c].values)
  input_test_list.append(test[c].values)

In [ ]:
# input models will capture all the input list
input_models=[]
# output embeddings will capture all the output embeddings
output_embeddings=[]
for c in cat_cols:
  cat_emb_name= c+'_Embedding'
  #Identifying the number of unique values in the category
  no_of_unique_cat  = train[c].nunique()
  embedding_size = 32 #as per Research Paper
  input_model = Input(shape=(1,),name = c+'_Input')
  output_model = Embedding(no_of_unique_cat, embedding_size,name=cat_emb_name)(input_model)
  output_model = Reshape(target_shape=(embedding_size,))(output_model)   

  input_models.append(input_model)
  output_embeddings.append(output_model) 

conc = Concatenate()(output_embeddings)
dense1 = Dense(256, activation='relu')(conc)
dense2 = Dense(256, activation='relu', )(dense1)
drop1 = Dropout(0.5)(dense2)
dense3 = Dense(256, activation='relu', )(drop1)
drop2 = Dropout(0.5)(dense3)
dense4 = Dense(256, activation='relu', )(drop2)
drop3 = Dropout(0.5)(dense4)
dense5 = Dense(256, activation='relu', )(drop3)
#avg_pool = keras.layers.AveragePooling1D(pool_size=(4,4),strides=(1,1), padding='same')(dense5)
outs = Dense(1, activation='sigmoid')(dense5)

model = keras.Model(inputs=input_models, outputs=outs)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
C1_Input (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
C2_Input (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
C3_Input (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
C4_Input (InputLayer)           [(None, 1)]          0                                            
______________________________________________________________________________________________

In [ ]:
opt = keras.optimizers.Adam(learning_rate=3e-5)
model.compile(optimizer=opt,loss='binary_crossentropy',metrics=[tf.keras.metrics.AUC(from_logits=True)])

In [ ]:
history = model.fit(input_train_list, train['label'],validation_data=(input_test_list, test['label']), batch_size = 256,epochs =  10 )

Epoch 1/10
313/313 [==============================] - 19s 37ms/step - loss: 0.5645 - auc: 0.5281 - val_loss: 0.5316 - val_auc: 0.6410
Epoch 2/10
313/313 [==============================] - 11s 35ms/step - loss: 0.5081 - auc: 0.6537 - val_loss: 0.5129 - val_auc: 0.6815
Epoch 3/10
313/313 [==============================] - 11s 35ms/step - loss: 0.4861 - auc: 0.7092 - val_loss: 0.5020 - val_auc: 0.7013
Epoch 4/10
313/313 [==============================] - 11s 35ms/step - loss: 0.4641 - auc: 0.7544 - val_loss: 0.4944 - val_auc: 0.7142
Epoch 5/10
313/313 [==============================] - 11s 35ms/step - loss: 0.4368 - auc: 0.7974 - val_loss: 0.4929 - val_auc: 0.7196
Epoch 6/10
313/313 [==============================] - 11s 35ms/step - loss: 0.4038 - auc: 0.8400 - val_loss: 0.5036 - val_auc: 0.7156
Epoch 7/10
313/313 [==============================] - 11s 34ms/step - loss: 0.3654 - auc: 0.8766 - val_loss: 0.5305 - val_auc: 0.7094
Epoch 8/10
313/313 [==============================] - 11s 35ms

In [ ]:
#average auc
np.mean(history.history['auc'])

0.7927197873592376